In [ ]:
from googleapiclient.discovery import build
from datetime import datetime
import pandas as pd

# 您的 API 金鑰
API_KEY = "api"

# 初始化 YouTube API 客戶端
youtube = build('youtube', 'v3', developerKey=API_KEY)

# 搜尋包含關鍵字「XXX」的影片
關鍵字 = "XXX"

# 定義時間範圍，僅使用日期部分
開始日期 = datetime(2018, 7, 1).date().strftime('%Y-%m-%d')  # 從 2018 年 7 月 1 日開始
結束日期 = datetime(2024, 7, 1).date().strftime('%Y-%m-%d')  # 到 2024 年 7 月 1 日結束

# 執行搜尋請求，按發佈時間排序（由舊到新），最多 500 筆結果
搜尋請求 = youtube.search().list(
    q=關鍵字,
    part='snippet',
    type='video',
    publishedAfter=開始日期 + 'T00:00:00Z',  # 加上時間部分
    publishedBefore=結束日期 + 'T23:59:59Z',   # 加上時間部分
    maxResults=1000,  # 一次最多 1000 筆結果
    order='date',  # 按照發佈時間排序
)

# 執行第一頁請求
搜尋回應 = 搜尋請求.execute()

# 準備資料準備匯出到 Excel
影片資料 = []
while 搜尋回應:
    for item in 搜尋回應['items']:
        影片標題 = item['snippet']['title']
        影片發佈日期 = item['snippet']['publishedAt'].split('T')[0]  # 只取日期部分
        影片網址 = f"https://www.youtube.com/watch?v={item['id']['videoId']}"

        影片資料.append({
            '標題': 影片標題,
            '發佈日期': 影片發佈日期,
            '網址': 影片網址
        })

    # 檢查是否有下一頁
    if 'nextPageToken' in 搜尋回應:
        下一頁代碼 = 搜尋回應['nextPageToken']
        搜尋回應 = youtube.search().list(
            q=關鍵字,
            part='snippet',
            type='video',
            publishedAfter=開始日期 + 'T00:00:00Z',  # 加上時間部分
            publishedBefore=結束日期 + 'T23:59:59Z',   # 加上時間部分
            maxResults=500,
            order='date',
            pageToken=下一頁代碼
        ).execute()
    else:
        break

# 轉換為 DataFrame
df = pd.DataFrame(影片資料)

# 排序 DataFrame 按照發佈時間（由舊到新）
df = df.sort_values(by='發佈日期').reset_index(drop=True)

# 儲存到 Excel
檔案名稱 = '/Users//Documents/youtube_furl_details.xlsx'
df.to_excel(檔案名稱, index=False)

print(f"資料已按發佈日期由舊到新排序並儲存至 {檔案名稱}")




